In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
import requests
import random
import xlrd
import csv
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib as mpl
import matplotlib.style
import seaborn as sns  
import matplotlib.pyplot as plt

In [4]:
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

In [5]:
# setting to default parameters
plt.rcParams.update(plt.rcParamsDefault)

# formatting for decimal places
pd.set_option("display.float_format", "{:.2f}".format)


In [6]:

# matplotlib settings
mpl.rcParams.update(mpl.rcParamsDefault)
#plt.style.use('whitegrid')
mpl.rcParams["figure.figsize"] = (12, 8)
mpl.rcParams["axes.grid"] = False

In [7]:
# setting seed for model reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

In [8]:
# setting the destination for the data folder
path = os.path.join(os.getcwd(), "C:/Users/ASUS/Petroleum-Production-Engineering/src/data")
norm_path = os.path.normpath(path)

In [9]:
# defining a function to scrape NDIC data
# https://www.dmr.nd.gov/oilgas/
# data from May 2015 to December 2018 will be used as a training dataset
# data from 2019 will be used as a test dataset

In [ ]:
# function to scrape data from NDIC
def scrape_ndic(months_list):
    '''function to scrape NDIC data'''
    # link to website with production data
    website = "https://www.dmr.nd.gov/oilgas/mpr/"
    df = pd.DataFrame()
    # loop through all of the dates in the list
    for period in months_list:
        url = website + period + ".xlsx"
        req = requests.get(url)
        book = xlrd.open_workbook(file_contents=req.content)
        sheet = book.sheet_by_index(0)
        for i in range(1, sheet.nrows):
            temp_value = sheet.cell_value(i, 0)
            year, month, day, hour, minute, second = xlrd.xldate_as_tuple(temp_value, book.datemode)
            sheet._cell_values[i][0] = datetime(year, month, 1).strftime("%m/%Y")
        new_file = (path + '\\'+ period + ".csv")
        csv_file = open(new_file, "w", newline="")
        writer = csv.writer(csv_file)
        # iteration through each row for data pull
        for rownum in range(sheet.nrows):
            writer.writerow(sheet.row_values(rownum))
        csv_file.close()
        df = pd.read_csv(new_file)
        df = df.append(df)
    # dataframe with entire monthly production
    return df


In [10]:
from openpyxl import load_workbook
from io import BytesIO

In [11]:
import logging
import time

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("scraping.log", mode='a', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

In [12]:
def scrape_ndic(months_list, path="."):
    """
    Function to scrape monthly NDIC production data
    and return a combined DataFrame.
    """
    logging.info("Web scraping session started. Total URLs: %d", len(months_list))
    website = "https://www.dmr.nd.gov/oilgas/mpr/"
    all_dfs = []

    for period in months_list:
        # Download XLSX file
        url = website + period + ".xlsx"
        req = requests.get(url)
        req.raise_for_status()

        # Load workbook from memory
        book = load_workbook(filename=BytesIO(req.content), read_only=True)
        sheet = book.active

        # Convert worksheet into list of lists
        data = []
        for row in sheet.iter_rows(values_only=True):
            data.append(list(row))

        # Transform into DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])  # first row = headers

        # Ensure first column is Month/Year (if it is a date in Excel)
        if isinstance(df.iloc[0, 0], datetime):
            df.iloc[:, 0] = df.iloc[:, 0].apply(lambda d: datetime(d.year, d.month, 1).strftime("%m/%Y"))

        # Save to CSV if desired
        new_file = f"C:/Users/ASUS/Petroleum-Production-Engineering/src/data/{period}.csv"
        df.to_csv(new_file, index=False)
        logging.info('Scraped %s successfully.', period)

        all_dfs.append(df)

    # Concatenate all months into one DataFrame
    final_df = pd.concat(all_dfs, ignore_index=True)
    logging.info("Web scraping session finished.")

    return final_df

In [13]:
train_list = ["2015_05", "2015_06"]

In [ ]:
train_prod_data = scrape_ndic(train_list)
train_prod_data["ReportDate"] = pd.to_datetime(train_prod_data["ReportDate"])
train_prod_data.to_csv("C:/Users/ASUS/Petroleum-Production-Engineering/src/data/train_prod.csv")

In [14]:
train_prod = pd.read_csv('train_prod.csv')

In [ ]:
def pre_process(df, column):
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.info()
    print(df.columns)
    # descriptive statistics
    df.describe().T
    df.head(15)
    df.nunique()    
    df.dtypes
    df.shape
    # filtering
    df.dropna(inplace=True)
    # drop rows where oil rate is 0
    df = df[(df[column].notnull()) & (df[column] > 0)]
    return df

In [ ]:
train_prod = pre_process(train_prod, 'Oil')

In [16]:
train_prod.head(20)

,Unnamed: 0,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
0,0,2015-05-01,33053043310000,23615,MUREX PETROLEUM CORPORATION,AMBER ELIZABETH 36-25H,SESW,36,151,101,...,BAKKEN,1525.00,1335.00,29.00,1686,1739.00,1519,0.00,47.85,-103.55
1,1,2015-05-01,33053038990000,22021,STATOIL OIL & GAS LP,BILL 14-23 2TFH,SWSW,11,151,101,...,BAKKEN,1410.00,5805.00,31.00,1370,1721.00,1720,1.00,47.91,-103.58
2,2,2015-05-01,33053048330000,25091,STATOIL OIL & GAS LP,BILL 14-23 3H,NWNE,14,151,101,...,BAKKEN,5035.00,12773.00,28.00,5087,3754.00,3249,505.00,47.90,-103.57
3,3,2015-05-01,33053050010000,25645,STATOIL OIL & GAS LP,BILL 14-23 4TFH,NWNE,14,151,101,...,BAKKEN,2419.00,17671.00,31.00,2287,232.00,201,31.00,47.90,-103.57
4,4,2015-05-01,33053048340000,25092,STATOIL OIL & GAS LP,BILL 14-23 5TFH,NWNE,14,151,101,...,BAKKEN,771.00,17718.00,31.00,729,53.00,46,7.00,47.90,-103.57
5,5,2015-05-01,33053050000000,25644,STATOIL OIL & GAS LP,BILL 14-23 6H,NWNE,14,151,101,...,BAKKEN,5558.00,11971.00,31.00,5316,5196.00,4497,699.00,47.90,-103.57
6,6,2015-05-01,33053039010000,22023,STATOIL OIL & GAS LP,BILL 14-23 1H,SWSW,11,151,101,...,BAKKEN,6848.00,6926.00,31.00,6748,8537.00,7791,746.00,47.91,-103.58
7,7,2015-05-01,33053042390000,23319,STATOIL OIL & GAS LP,PORTER 35-26 #1TFH,SESW,35,151,101,...,BAKKEN,1763.00,5903.00,31.00,1708,2141.00,2140,1.00,47.85,-103.58
8,8,2015-05-01,33053042400000,23320,STATOIL OIL & GAS LP,PORTER 35-26 #2H,SESW,35,151,101,...,BAKKEN,1973.00,3258.00,31.00,1925,2140.00,2139,1.00,47.85,-103.58
9,9,2015-05-01,33053042380000,23302,STATOIL OIL & GAS LP,TIMBER CREEK 13-24 1TFH,NENW,13,151,101,...,BAKKEN,3123.00,6513.00,30.00,3200,3159.00,2580,579.00,47.91,-103.55


In [17]:
train_prod.drop("Unnamed: 0", axis=1, inplace=True)

In [18]:
train_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28261 entries, 0 to 28260
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ReportDate  28261 non-null  object 
 1   API_WELLNO  28261 non-null  int64  
 2   FileNo      28261 non-null  int64  
 3   Company     28261 non-null  object 
 4   WellName    28261 non-null  object 
 5   Quarter     28261 non-null  object 
 6   Section     28261 non-null  int64  
 7   Township    28261 non-null  int64  
 8   Range       28261 non-null  int64  
 9   County      28261 non-null  object 
 10  FieldName   28261 non-null  object 
 11  Pool        28261 non-null  object 
 12  Oil         27954 non-null  float64
 13  Wtr         27954 non-null  float64
 14  Days        27954 non-null  float64
 15  Runs        28261 non-null  int64  
 16  Gas         27954 non-null  float64
 17  GasSold     28261 non-null  int64  
 18  Flared      27954 non-null  float64
 19  Lat         28261 non-nul

In [19]:
train_prod.describe().T

,count,mean,std,min,25%,50%,75%,max
API_WELLNO,28261.00,33050747635324.30,30873085261.55,33007000140000.00,33025007150000.00,33053041490000.00,33061024110000.00,33105039270000.00
FileNo,28261.00,19767.71,6592.53,35.00,16284.00,20688.00,24953.00,30755.00
Section,28261.00,18.31,10.46,1.00,9.00,18.00,27.00,36.00
Township,28261.00,151.68,7.43,129.00,148.00,152.00,156.00,164.00
Range,28261.00,95.57,5.78,76.00,93.00,96.00,100.00,107.00
Oil,27954.00,2482.60,3429.94,-52.00,314.00,1474.00,3169.75,40368.00
Wtr,27954.00,2855.12,5618.78,-1.00,318.00,1318.50,3089.00,221085.00
Days,27954.00,24.47,10.68,0.00,24.00,30.00,31.00,31.00
Runs,28261.00,2585.33,3650.72,0.00,321.00,1512.00,3244.00,47528.00
Gas,27954.00,3406.40,6061.05,0.00,231.00,1540.00,3975.75,217393.00


In [20]:
train_prod.nunique()

ReportDate        2
API_WELLNO    14084
FileNo        14084
Company         154
WellName      14073
Quarter          81
Section          36
Township         35
Range            32
County           16
FieldName       514
Pool             35
Oil            7619
Wtr            7856
Days             32
Runs           7783
Gas            8912
GasSold        8125
Flared         3337
Lat           13880
Long          13801
dtype: int64

In [22]:
train_prod.shape

(28261, 21)

In [23]:
train_prod.dropna(inplace=True)

In [ ]:
train_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27954 entries, 0 to 28260
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ReportDate  27954 non-null  object 
 1   API_WELLNO  27954 non-null  int64  
 2   FileNo      27954 non-null  int64  
 3   Company     27954 non-null  object 
 4   WellName    27954 non-null  object 
 5   Quarter     27954 non-null  object 
 6   Section     27954 non-null  int64  
 7   Township    27954 non-null  int64  
 8   Range       27954 non-null  int64  
 9   County      27954 non-null  object 
 10  FieldName   27954 non-null  object 
 11  Pool        27954 non-null  object 
 12  Oil         27954 non-null  float64
 13  Wtr         27954 non-null  float64
 14  Days        27954 non-null  float64
 15  Runs        27954 non-null  int64  
 16  Gas         27954 non-null  float64
 17  GasSold     27954 non-null  int64  
 18  Flared      27954 non-null  float64
 19  Lat         27954 non-null  fl

In [25]:
train_prod = train_prod[(train_prod["Oil"].notnull()) & (train_prod["Oil"] > 0)]

In [27]:
train_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24437 entries, 0 to 28259
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ReportDate  24437 non-null  object 
 1   API_WELLNO  24437 non-null  int64  
 2   FileNo      24437 non-null  int64  
 3   Company     24437 non-null  object 
 4   WellName    24437 non-null  object 
 5   Quarter     24437 non-null  object 
 6   Section     24437 non-null  int64  
 7   Township    24437 non-null  int64  
 8   Range       24437 non-null  int64  
 9   County      24437 non-null  object 
 10  FieldName   24437 non-null  object 
 11  Pool        24437 non-null  object 
 12  Oil         24437 non-null  float64
 13  Wtr         24437 non-null  float64
 14  Days        24437 non-null  float64
 15  Runs        24437 non-null  int64  
 16  Gas         24437 non-null  float64
 17  GasSold     24437 non-null  int64  
 18  Flared      24437 non-null  float64
 19  Lat         24437 non-null  fl

In [28]:
def plot_production_rate(df):
    '''Plot decline curve using production rates'''
    sns.lineplot(x = df['ReportDate'], y = df['oil_rate'], markers=True, dashes=False, 
                 label="Oil Production",color='blue', linewidth=1.5)
    plt.title('Decline Curve', fontweight='bold', fontsize = 20)
    plt.xlabel('Time', fontweight='bold', fontsize = 15)
    plt.ylabel('Oil Production Rate (bbl/d)', fontweight='bold', fontsize = 15)
    plt.show()

In [29]:
def decline_curve(curve_type, q_i):
    if curve_type == "exponential":

        def exponential_decline(T, d):
            return q_i * np.exp(-d * T)
        return exponential_decline

    elif curve_type == "hyperbolic":

        def hyperbolic_decline(T, d_i, b):
            return q_i / np.power((1 + b * d_i * T), 1.0 / b)
        return hyperbolic_decline

    elif curve_type == "harmonic":

        def parabolic_decline(T, d_i):
            return q_i / (1 + d_i * T)
        return parabolic_decline

    else:
        raise "Unknown Decline Curve!"


def L2_norm(Q, Q_obs):
    return np.sum(np.power(np.subtract(Q, Q_obs), 2))

In [30]:
# convert time to datetime and set as dataframe index
train_prod["ReportDate"] = pd.to_datetime(train_prod["ReportDate"])

In [32]:

#bakken_data.set_index("ReportDate", inplace=True)
train_prod["First_Prod_Date"] = train_prod.groupby("API_WELLNO")["ReportDate"].transform('min')
train_prod["Days_Online"] = (train_prod["ReportDate"] - train_prod["First_Prod_Date"]).dt.days 


In [39]:
train_prod

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long,First_Prod_Date,Days_Online
0,2015-05-01,33053043310000,23615,MUREX PETROLEUM CORPORATION,AMBER ELIZABETH 36-25H,SESW,36,151,101,MCK,...,1335.00,29.00,1686,1739.00,1519,0.00,47.85,-103.55,2015-05-01,0
1,2015-05-01,33053038990000,22021,STATOIL OIL & GAS LP,BILL 14-23 2TFH,SWSW,11,151,101,MCK,...,5805.00,31.00,1370,1721.00,1720,1.00,47.91,-103.58,2015-05-01,0
2,2015-05-01,33053048330000,25091,STATOIL OIL & GAS LP,BILL 14-23 3H,NWNE,14,151,101,MCK,...,12773.00,28.00,5087,3754.00,3249,505.00,47.90,-103.57,2015-05-01,0
3,2015-05-01,33053050010000,25645,STATOIL OIL & GAS LP,BILL 14-23 4TFH,NWNE,14,151,101,MCK,...,17671.00,31.00,2287,232.00,201,31.00,47.90,-103.57,2015-05-01,0
4,2015-05-01,33053048340000,25092,STATOIL OIL & GAS LP,BILL 14-23 5TFH,NWNE,14,151,101,MCK,...,17718.00,31.00,729,53.00,46,7.00,47.90,-103.57,2015-05-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28250,2015-06-01,33089008080000,26316,WHITING OIL AND GAS CORPORATION,RIDL FEDERAL 44-12PH,SWSE,12,139,99,STK,...,1120.00,29.00,2823,3029.00,2959,13.00,46.86,-103.11,2015-05-01,31
28251,2015-06-01,33089006140000,19926,WHITING OIL AND GAS CORPORATION,ROLLER 21-26TFH,NENW,26,139,99,STK,...,576.00,30.00,371,385.00,325,0.00,46.83,-103.14,2015-05-01,31
28255,2015-06-01,33089003290000,10752,"NEW MILLENNIUM RESOURCES, INC.",POLANCHEK 8-34,SENE,34,140,99,STK,...,2123.00,30.00,2256,30.00,0,0.00,46.90,-103.15,2015-05-01,31
28258,2015-06-01,33009012640000,6347,"ENERGYQUEST II, LLC",RICE 2,SENW,26,163,79,BOT,...,93.00,30.00,0,0.00,0,0.00,48.92,-100.88,2015-05-01,31


In [40]:
grouped_data = train_prod.groupby(['API_WELLNO']).sum()

TypeError: datetime64 type does not support sum operations